# _*Qiskit Aqua: Solving linear systems of equations with the HHL algorithm*_

***
### Contributors
David Bucher<sup>[1]</sup>, Jan Mueggenburg<sup>[1]</sup>, Gawel Kus<sup>[1]</sup>, Isabel Haide<sup>[1]</sup>, Shubha Deutschle<sup>[1]</sup>, Harry Barowski<sup>[1]</sup>, Dominik Steenken<sup>[1]</sup>, and Albert Frisch<sup>[1]</sup>
### Affiliation
- <sup>[1]</sup>IBMQ

The HHL algorithm (after the author’s surnames Harrow-Hassidim-Lloyd) [1] is a quantum algorithm to solve systems of linear equations $A \vec{x} = \vec{b}$. To perform this calculation quantum mechanically, we need in general 4 mainly steps requiring three qubit registers:
<ol>
<li>First, we have to express the vector $\vec{b}$ as a quantum state $|b\rangle$ on a quantum register.</li>
<li>Now, we have to decompose $\vec{b}$ into a superposition of eigenvectors of A remembering on the linear combination of the vector $\vec{b}$. We achieve this using the Quantum Phase Estimation algorithm (Quantum Phase Estimation (QPE)). Since the matrix is hereby diagonalized wherefore $A$ is easily invertible.</li>
<li>The inversion of the eigenvector base of $A$ is achieved by rotating an ancillary qubit by an angle $\arcsin \left( \frac{C}{\lambda _{\text{i}}} \right)$ around the y-axis where $\lambda_{\text{i}}$ are the eigenvalues of $A$. Now, we obtain the state $A^{-1}|b\rangle = |x \rangle$.</li>
<li>We need to uncompute the register storing the eigenvalues using the inverse QPE. We measure the ancillary qubit whereby the measurement of 1 indicates that the matrix inversion was successfully. The inverse QPE leaves the system in a state proportional to the solution vector $|x\rangle$. In many cases one is not interested in the single vector elements of $|x\rangle$ but only on certain properties. These are accessible by applying a problem-specific operator $M$ to the state $|x\rangle$. Another use-case of the HHL algorithm is the implementation in a larger quantum program.</li>
</ol>
Currently only hermitian matrices with a dimension of $2^n$ are supported.

Take into account that in the general case, the entries of $\vec{x}$ can not be efficiently read out because we would need to know all coefficients describing the quantum state.
In the following examples, we ignore this constraint and show for our small linear system as a proof of principle that $\vec{x}$ is calculated correctly.

References:
- A. W. Harrow, A. Hassidim, and S. Lloyd, Phys. Rev. Lett. 103, 150502 (2009), e-print arXiv 0811.3171
- S. Barz, I. Kassal, M. Ringbauer, Y. Ole Lipp, B. Dakić, A. Aspuru-Guzik, and P. Walther, Sci Rep. 4: 6115 (2014), e-print arXiv 1302.1210

In [ ]:
import sys
sys.path.insert(0, '/home/AlbertFrisch/qiskit/qiskit-terra')
#sys.path.insert(0, '/home/shubha.deutschle/qiskit/qiskit-terra')
#print(sys.path)

In [ ]:
from qiskit.aqua import run_algorithm
from qiskit.aqua.input import LinearSystemInput
import numpy as np

In [ ]:
params = {
    'problem': {
        'name': 'linear_system'
    },
    'algorithm': {
        'name': 'HHL'
    },
    'eigs': {
        'expansion_mode': 'suzuki',
        'expansion_order': 2,
        'name': 'EigsQPE',
        'num_ancillae': 3,
        'num_time_slices': 50
    },
    'reciprocal': {
        'name': 'Lookup'
    },
    'backend': {
        'name': 'statevector_simulator'
    }
}

## 2x2 diagonal matrix

We show an example of solving a linear system of equations including the diagonal matrix
$$
M=
\begin{bmatrix}
1 & 0 \\
0 & 2
\end{bmatrix}$$ and the vector $$\vec{b}= \left( \begin{array}{c}1 \\ 4  \end{array} \right)$$
The solution which is obtained by the classical algorithm is also shown in the output. The fidelity between the solution of hhl and the classical solution depends on the equations to solve, it is also given in the output. Furthermore, the probability that HHL was running successfully, is also given in the output.

In [ ]:
matrix = [[1, 0], [0, 2]]
vector = [1, 4]
params['input'] = {
    'name': 'LinearSystemInput',
    'matrix': matrix,
    'vector': vector
}

In [ ]:
result = run_algorithm(params)
print("hhl solution ", result['solution_hhl'])
print("classical solution", result['solution_classical'])
fidelity_hhl_to_classical = result['fidelity_hhl_to_classical']
print("fidelity", result['fidelity_hhl_to_classical'])
print("probability", result['probability_result'])

The above example is using the smallest possibly eigenvalue of the inserted matrix M for calculation, because we did no additional definition for the $\text{scale}$ variable. The probabilty that hhl works successfully depends on the $\text{scale}$ variable and increases with higher $\text{scale}$ ($\text{scale}$ is defined as $\text{scale}$ $\in [0,1]$). In the following, we define $\text{scale}=0.5$ and show how the results are influenced thereby.

In [ ]:
params['reciprocal'] = {    
    'scale': 0.5
}
result = run_algorithm(params)
print("hhl solution ", result['solution_hhl'])
print("classical solution", result['solution_classical'])
print("fidelity", result['fidelity_hhl_to_classical'])
print("probability", result['probability_result'])

## 2x2 non-diagonal matrix

Below is an example of the HHL solution coming from a non-diagonal matrix
$$
M=
\begin{bmatrix}
1 & 3 \\
3 & 2
\end{bmatrix}$$ and a vector $$\vec{b}= \left( \begin{array}{c}1 \\ 1  \end{array} \right)$$


In [ ]:
matrix = [[1, 3], [3, 2]]
vector = [1, 1]
params['input'] = {
    'name': 'LinearSystemInput',
    'matrix': matrix,
    'vector': vector
}

In [ ]:
result = run_algorithm(params)
print("hhl solution ", result['solution_hhl'])
print("classical solution", result['solution_classical'])
print("fidelity", result['fidelity_hhl_to_classical'])
print("probability", result['probability_result'])

If you want to check how many gates are allpied by HHL on the qubit which passes the most quantity of gates (circuit depth) or if you want to know how many qubits are required (circuit width), you can print it out by

In [ ]:
print("circuit_depth", result['circuit_depth'])
print("circuit_width", result['circuit_width'])

## 8x8 non-diagonal matrix

For a higher dimensional matrix, we show in the following the HHl solution of a linear equation system consisting of the 8x8 dimensional matrix
        
        
$$
M=
\begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix}$$ and the vector $$\vec{b}= \left( \begin{array}{c}1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \end{array} \right)$$

In [ ]:
matrix = [[1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1]]
vector = [1, 0, 0, 0, 0, 0, 0, 1]
params['input'] = {
    'name': 'LinearSystemInput',
    'matrix': matrix,
    'vector': vector
}

In [ ]:
result = run_algorithm(params)
print("hhl solution ", result['solution_hhl'])
print("classical solution", result['solution_classical'])
print("fidelity", result['fidelity_hhl_to_classical'])
print("probability", result['probability_result'])

## 4x4 randomly-generated matrix

Now, we show the application of HHL on an arbitrary 4x4 Matrix. We fix the arbitrary matrix entries achieving reproducibility of the HHL run. We choose $$\vec{b}= \left( \begin{array}{c}1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \end{array} \right)$$

In [ ]:
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms.single_sample import HHL
from qiskit.aqua.utils import random_hermitian

In [ ]:
params2 = {
    "algorithm": {
        "name": "HHL"
    },
    "backend": {
        "basis_gates": None,
        "coupling_map": None,
        "initial_layout": None,
        "max_credits": 10,
        "name": "statevector_simulator",
        "provider": "qiskit.Aer",
        "shots": 10,
        "timeout": None,
        "wait": 5.0
    },
    "eigs": {
        "evo_time": None,
        "expansion_mode": "suzuki",
        "expansion_order": 2,
        "name": "EigsQPE",
        "num_ancillae": 5,
        "negative_evals": False,
        "num_time_slices": 50
    },
    "initial_state": {
        "name": "CUSTOM",
        "state": "zero",
        "state_vector": None
    },
    "iqft": {
        "name": "STANDARD"
    },
    "problem": {
        "circuit_cache_file": None,
        "circuit_caching": False,
        "name": "linear_system",
        "random_seed": None,
        "skip_qobj_deepcopy": False,
        "skip_qobj_validation": False
    },
    "qft": {
        "name": "STANDARD"
    },
    "reciprocal": {
        "evo_time": None,
        "lambda_min": None,
        "name": "Lookup",
        "pat_length": None,
        "subpat_length": None
    }
}

In [ ]:
# set the random seed to get the same pseudo-random matrix for every run
np.random.seed(1)
matrix = random_hermitian(4)
vector = [1, 2, 3, 1]

algo_input = LinearSystemInput(matrix=matrix, vector=vector)
hhl = HHL.init_params(params2, algo_input)
backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=backend)
result_hhl = hhl.run(quantum_instance)

print("input matrix:")
print(np.array_str(result_hhl['input_matrix']))

print("HHL results:")
print("hhl solution ", result_hhl['solution_hhl'])
print("classical solution", result_hhl['solution_classical'])
print("fidelity", result_hhl['fidelity_hhl_to_classical'])
print("probability", result_hhl['probability_result'])